In [1]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import pandas as pd
import awswrangler as wr

In [ ]:
# pip install awswrangler

# 1. load data from S3

In [2]:
order = wr.s3.read_csv(path="s3://bucket-to-snowflake/orders.csv")
op_train = wr.s3.read_csv(path='s3://bucket-to-snowflake/order_products__train.csv.gz', compression='gzip')
op_prior = wr.s3.read_csv(path='s3://bucket-to-snowflake/order_products__prior.csv.gz', compression='gzip')
product = wr.s3.read_csv(path='s3://bucket-to-snowflake/products.csv')

# 2. reshape data

In [3]:
# change data type
order['eval_set'] = order['eval_set'].astype('category')
product['product_name'] = product['product_name'].astype('category')

In [ ]:
order_opp_merge = order.merge(op_prior, on = 'order_id', how = 'inner')

In [ ]:
# user predictor 
user = order_opp_merge.groupby('user_id')['order_number'].max().to_frame('user_total_orders')

22:15 NOV 30 2022